In [9]:
from langchain_community.document_loaders import DirectoryLoader, PyPDFLoader
loader = DirectoryLoader(r'D:\MSHack\files\files', glob="*.pdf", loader_cls=PyPDFLoader, silent_errors=True)
pages = loader.load_and_split()


Ignoring wrong pointing object 14 0 (offset 0)
Ignoring wrong pointing object 16 0 (offset 0)
Ignoring wrong pointing object 18 0 (offset 0)
Ignoring wrong pointing object 30 0 (offset 0)
Ignoring wrong pointing object 8 0 (offset 0)
Ignoring wrong pointing object 12 0 (offset 0)
Ignoring wrong pointing object 47 0 (offset 0)
Ignoring wrong pointing object 49 0 (offset 0)
Ignoring wrong pointing object 9 0 (offset 0)
Ignoring wrong pointing object 11 0 (offset 0)
Ignoring wrong pointing object 16 0 (offset 0)
Ignoring wrong pointing object 18 0 (offset 0)
Ignoring wrong pointing object 20 0 (offset 0)
Ignoring wrong pointing object 37 0 (offset 0)
Ignoring wrong pointing object 40 0 (offset 0)
Ignoring wrong pointing object 42 0 (offset 0)
Ignoring wrong pointing object 45 0 (offset 0)
Ignoring wrong pointing object 55 0 (offset 0)
Ignoring wrong pointing object 57 0 (offset 0)
Ignoring wrong pointing object 73 0 (offset 0)
Ignoring wrong pointing object 75 0 (offset 0)
Ignoring wrong 

In [10]:
len(pages)

669

In [11]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size= 1024,
    chunk_overlap= 100 , 
    length_function= len ,
    is_separator_regex= False
)

In [12]:
texts = text_splitter.create_documents([pages[i].page_content for i in range(len(pages))])

In [16]:
from qdrant_client import QdrantClient
from qdrant_client.models import Distance, VectorParams

client = QdrantClient(url="http://localhost:6333")

client.create_collection(
    collection_name="MedicalPapers",
    vectors_config=VectorParams(size=1024, distance=Distance.DOT),
)



True

In [17]:
import cohere
import qdrant_client
from qdrant_client.models import Batch

cohere_client = cohere.Client("eFUHV96luEGkkcHOkuQlHu7gxocJUOwvE4jfLI9T")
client = qdrant_client.QdrantClient()
client.upsert(
    collection_name="MedicalPapers",
    points=Batch(
        ids=range(len(texts)),
        vectors=cohere_client.embed(
            model="embed-english-v3.0",  # New Embed v3 model
            input_type="search_document",  # Input type for indexing documents
            texts= [texts[i].page_content for i in range(len(texts))],
        ).embeddings,
        payloads = [{"Context{}".format(index): value} for index, value in enumerate([texts[i].page_content for i in range(len(texts))], start=1)],
    ),
)


UpdateResult(operation_id=0, status=<UpdateStatus.COMPLETED: 'completed'>)